# Interact with decay data in ENDF-6 format (MF=8, MT=457)

First import `sandy` and other packages importing for formatting and postprocessing that are used in this notebook.

In [1]:
import sandy
import yaml

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read decay data in ENDF-6 format

As always, any nuclear data evaluation can be imported into a `Endf6` instance

In [2]:
tape = sandy.read_formatted_file("dec-027_Co_060.endf")

The radioactive decay data text (in ENDF-6 format) for all sections and isotopes in `tape` can be 
parsed and extracted in a hierarchical format (nested `dict`) into a `DecayData` instance, say, variable `rdd`.

In [3]:
rdd = sandy.DecayData.from_endf6(tape)
rdd.data

{270600: {'half_life': 166344200.0,
  'decay_constant': 4.166945289105032e-09,
  'decay_modes': {'10': {'decay_products': {280600: 1.0},
    'branching_ratio': 1.0}},
  'stable': False,
  'spin': 5.0,
  'parity': 1.0,
  'decay_energy': {'beta': 96769.1,
   'gamma': 2503844.0,
   'alpha': 0.0,
   'total': 2600613.1},
  'decay_energy_uncertainties': {'beta': 241.9428,
   'gamma': 352.1859,
   'alpha': 0.0}},
 270601: {'half_life': 628.02,
  'decay_constant': 0.0011037023989044065,
  'decay_modes': {'10': {'decay_products': {280600: 1.0},
    'branching_ratio': 0.0024},
   '30': {'decay_products': {270600: 1.0}, 'branching_ratio': 0.9976}},
  'stable': False,
  'spin': 2.0,
  'parity': 1.0,
  'decay_energy': {'beta': 56430.24,
   'gamma': 6614.704,
   'alpha': 0.0,
   'total': 63044.943999999996},
  'decay_energy_uncertainties': {'beta': 8461.567,
   'gamma': 425.0476,
   'alpha': 0.0}}}

A better rendering of the `rdd` data content can be obtained with the `yaml` python package.

In [4]:
print(yaml.dump(rdd.data))

270600:
  decay_constant: 4.166945289105032e-09
  decay_energy:
    alpha: 0.0
    beta: 96769.1
    gamma: 2503844.0
    total: 2600613.1
  decay_energy_uncertainties:
    alpha: 0.0
    beta: 241.9428
    gamma: 352.1859
  decay_modes:
    '10':
      branching_ratio: 1.0
      decay_products:
        280600: 1.0
  half_life: 166344200.0
  parity: 1.0
  spin: 5.0
  stable: false
270601:
  decay_constant: 0.0011037023989044065
  decay_energy:
    alpha: 0.0
    beta: 56430.24
    gamma: 6614.704
    total: 63044.943999999996
  decay_energy_uncertainties:
    alpha: 0.0
    beta: 8461.567
    gamma: 425.0476
  decay_modes:
    '10':
      branching_ratio: 0.0024
      decay_products:
        280600: 1.0
    '30':
      branching_ratio: 0.9976
      decay_products:
        270600: 1.0
  half_life: 628.02
  parity: 1.0
  spin: 2.0
  stable: false



The description of the `rdd.data` structure is explained below, where `zam` and `zap` are 
equal to `Z * 10000 + A * 10 + M` for the parent and daughter isotopes, respectively, with 
`Z`, `A`, and `M` being the charge, nucleons and metastate numbers.

```yaml
    zam :
        half_life : value in s
        decay_constant : value in 1/s
        stable : True/False
        decay_energy :
            alpha: value in eV
            beta: value in eV
            gamma: value in eV
            total: value in eV
        decay_energy_uncertainty :
            alpha: value in eV
            beta: value in eV
            gamma: value in eV
            total: value in eV
        decay_modes :
            rtyp : 
                decay_products : 
                    zap : yield
                    zap_2 : yield_2
                    ...
                branching_ratio : value
            rtyp_2 : ...
    zam_2 : ...
```

`rtyp` is a string storing a sequence of integers, each of which defining the 
several decay events covered by a given decay mode.
The list of individual decay events is accessible via variable `sandy.decay_modes`.

In [5]:
print(yaml.dump(sandy.decay_modes))

0: gamma
1: beta
2: e.c.
3: i.t.
4: alpha
5: n
6: s.f.
7: p



## Extract structured information into dataframes

We can extract the decay chains information (decay constant, branching ratio, yield, ...) into a `pandas.DataFrame`.

In [6]:
chains = rdd.get_decay_chains()
chains

,PARENT,DAUGHTER,YIELD,BR,LAMBDA
0,270600,270600,-1.0,1.0000,4.166945e-09
1,270600,280600,1.0,1.0000,4.166945e-09
4,270601,270600,1.0,0.9976,1.103702e-03
2,270601,270601,-1.0,1.0000,1.103702e-03
3,270601,280600,1.0,0.0024,1.103702e-03


Or get the B-matrix, the matrix of branching ratios used to produce...

In [12]:
bm = rdd.get_bmatrix()
bm

PARENT,270600,270601
DAUGHTER,,
270600,0.0,1.0
270601,0.0,0.0


...the Q-matrix, to get cumulative yields `Y` from independent yields `C`, according to `Y = Q C`. 

In [8]:
qm = rdd.get_qmatrix()
qm

PARENT,270600,270601
DAUGHTER,,
270600,1.000000e+00,1.0
270601,1.110223e-16,1.0


Another thing you can get is the transition matrix `T` used to solve the depletion equation 
for radioactive decay `dN\N = T N `.

In [9]:
tm = rdd.get_transition_matrix()
tm

PARENT,270600,270601
DAUGHTER,,
270600,-4.166945e-09,0.001101
270601,0.000000e+00,-0.001104


## Convert decay data into HDF5 format

We can write the content of `rdd` into a hdf5 file, say `"decay.hdf5"`, 
under a group namespace to identify the library, e.g. `"endfb_71"`.

In [10]:
h5filename = "decay.hdf5"
libname = "endfb_71"
rdd.to_hdf5("decay.hdf5", "endfb_71")

In [11]:
rdd2 = sandy.DecayData.from_hdf5(h5filename, libname)
assert rdd2.get_decay_chains().equals(chains)